In [0]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy import stats
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from keras.layers import LeakyReLU
from sklearn import svm
from keras.utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import seaborn as sb
from scipy import stats
import time
from scipy.stats import wilcoxon
from google.colab import files

In [0]:
pd.show_versions()

**Fetching Data which will be used for evaluating the base learners and the stacked model**

In [0]:
stack_train_test= 'https://raw.githubusercontent.com/rohandongare-nci/18120199-Data/master/Training%20the%20entire%20Stack/Train%20entire%20stack.csv'
sdss_stk = pd.read_csv(stack_train_test)

**NNMF**

**For Training and Testing the entire stack**

In [0]:
sdss=sdss_stk

**Removing camera related and ID values**

In [0]:
unwanted_columns = ['camcol','run','rerun','objid','specobjid','field']
sdss.drop(unwanted_columns, axis=1, inplace=True)
sdss.head()

**NMF for photometric data**

In [0]:
sdss_colorbands=sdss[['u','g','r','i','z']]

In [0]:
from sklearn.decomposition import NMF
nnmf_model = NMF(n_components=3, init='random', random_state=18120199)

In [0]:
nnmf_components = nnmf_model.fit_transform(sdss_colorbands)

In [0]:
#appending nmf components to the SDSS dataframe
sdss = pd.concat((sdss, pd.DataFrame(nnmf_components)), axis=1)

In [0]:
sdss.rename({0: 'color_component_1', 1: 'color_component_2', 2: 'color_component_3'}, axis=1, inplace = True)

In [0]:
sdss_colour_c=sdss[['color_component_1','color_component_2','color_component_3']]

In [0]:
enc = LabelEncoder()
X=sdss[['ra','dec','color_component_1','color_component_2','color_component_3','redshift','fiberid','plate','mjd']]
y = enc.fit_transform(sdss['class'])
X.head()

**Power Transformation using Yeo-Johnson Transformer**

In [0]:
power_transform_yj = preprocessing.PowerTransformer(method='yeo-johnson', standardize=True)

In [0]:
X=power_transform_yj.fit_transform(X)

**Data Normalization**

In [0]:
X= preprocessing.normalize(X, norm='l2')

In [0]:
from collections import Counter
Counter(y)

**Undersampling data**

In [0]:
from imblearn.under_sampling import RandomUnderSampler

In [0]:
undersampler = RandomUnderSampler(random_state=18120199,replacement=False)
X_under, y_under = undersampler.fit_resample(X, y)

In [0]:
Counter(y_under)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X_under, y_under, test_size = 0.25, random_state = 18120199)

**Stratified K-Fold**

In [0]:
k = 10
str_kf = StratifiedKFold(n_splits=k, random_state=18120199, shuffle=False)

**Defining a function for stacking the models**

In [0]:
def get_stack_pred(classifier):
  #Initial train-test split- train sample will be used for k-fold, test sample will be used as the holdout set
  x_train, x_test, y_train, y_test = train_test_split(X_under, y_under, test_size = 0.25, random_state = 18120199)
  #initializing arrays to store predictions
  if_test=np.empty((x_train.shape[0],))
  oof_kfold = np.empty((k, x_test.shape[0]))
  infoldx_train=[]
  infoldy_train=[]
  infold_test=[]
  infoldy_test=[]
  #assigning train-test indices for every fold
  for j, (train_index, test_index) in enumerate(str_kf.split(x_train, y_train)):
    infoldx_train = x_train[train_index]
    infoldy_train = y_train[train_index]
    infoldx_test   = x_train[test_index]
    infoldy_test  = y_train[test_index]
    #print(infoldx_test.shape[0])
    #training the classifier on the infold train/test set 
    classifier.fit(infoldx_train,infoldy_train)
    #infold test prediction
    if_test[test_index]=classifier.predict(infoldx_test)
    infold_pred=classifier.predict(infoldx_test)
    #Infold classifier performance
    infold_accuracy = metrics.accuracy_score(infold_pred, infoldy_test)
    print(f"In Fold classifier accuracy: {infold_accuracy}")
    #holdout fold test prediction
    pred2=classifier.predict(x_test)
    #holdout set classifier performance
    out_of_fold_accuracy = metrics.accuracy_score(pred2,y_test)
    print(f"Out Of Fold accuracy: {out_of_fold_accuracy}")
    #adding predictions of the holdout set for every fold row-wise to an array
    oof_kfold[j,:]=classifier.predict(x_test)
  #taking mode of the holdout set predictions of each row
  mode=stats.mode(oof_kfold,axis=0)
  oof_kfold=mode[0]
  '''returning transposed columns so that predictions of each base learner 
  will later after appending them will have its own column'''
  return if_test.reshape(-1,1), oof_kfold.reshape(-1,1)

**Keras Classifier**

In [0]:
# Function to create model, required for KerasClassifier
def base_model():
  model_sdss_base = Sequential()
  model_sdss_base.add(Dense(1024, input_shape=(9,),kernel_initializer='random_uniform'))
  model_sdss_base.add(LeakyReLU(alpha=0.001))
  model_sdss_base.add(Dropout(0.2))
  model_sdss_base.add(Dense(512))
  model_sdss_base.add(LeakyReLU(alpha=0.001))
  model_sdss_base.add(Dropout(0.2))
  model_sdss_base.add(Dense(512))
  model_sdss_base.add(LeakyReLU(alpha=0.001))
  model_sdss_base.add(Dropout(0.2))
  adamopt=optimizers.Adam(lr=10**-3)
  model_sdss_base.add(Dense(3, activation='softmax'))
  model_sdss_base.compile(loss='categorical_crossentropy',
                    optimizer=adamopt, 
                    metrics=['accuracy'])
  return model_sdss_base

In [0]:
nn_els = EarlyStopping(monitor='acc', mode='max', verbose=1, patience=5)
nn_mck = ModelCheckpoint('best_observed_model.h5', monitor='acc', mode='max', verbose=1, save_best_only=True)
nn_model = KerasClassifier(build_fn=base_model, epochs=50, batch_size=300, verbose=1,callbacks=[nn_els,nn_mck])

In [0]:
nn_train, nn_test = get_stack_pred(nn_model)

In [0]:
!ls

In [0]:
#Downloading the best weights
from google.colab import files
files.download('best_observed_model.h5') 

In [0]:
class_names = ['Galaxy', 'Quasar', 'Star']

In [0]:
start_nn_base=time.time()
nn_train, nn_test = get_stack_pred(nn_model)
stop_nn_base=time.time()

In [0]:
from sklearn.metrics import confusion_matrix
cm_nn = confusion_matrix(Y_test, nn_test)
print(cm_nn)

In [0]:
print(classification_report(Y_test, nn_test,target_names=class_names))

In [0]:
df_nn_cm = pd.DataFrame(cm_nn,index=['Galaxy','Quasar','Star'],
                  columns = ['Galaxy','Quasar','Star'])
sb.heatmap(df_nn_cm, annot=True,fmt="g",cmap="Blues")

**Binomial test to check if the output of the base learner was not a result of pure chance**

In [0]:
stats.binom_test(58870, n=60615, p=0.973)

In [0]:
stats.binom_test(58870, n=60615, p=0.97,alternative="greater")

**Extra Tree Classifier**

In [0]:
from sklearn.ensemble import ExtraTreesClassifier

In [0]:
et_base = ExtraTreesClassifier(n_estimators=103,max_features= 0.96,criterion= 'entropy',min_samples_split= 2,max_depth= 50, min_samples_leaf= 1,bootstrap=True)      

In [0]:
start_et=time.time()
et_train, et_test=get_stack_pred(et_base)
end_et=time.time()

In [0]:
et_time=end_et-start_et
print(et_time)

In [0]:
from sklearn.metrics import confusion_matrix
cm_et = confusion_matrix(Y_test, et_test)
print(cm_et)

In [0]:
df_et_cm = pd.DataFrame(cm_et,index=['Galaxy','Quasar','Star'],
                  columns = ['Galaxy','Quasar','Star'])
sb.heatmap(df_et_cm, annot=True,fmt="g",cmap="Blues")

In [0]:
class_name = ['Galaxy', 'Quasar', 'Star']
print(classification_report(Y_test, et_test,target_names=class_name))

In [0]:
#Two-tailed binomial test to check if the results acheived were not due to chance alone
stats.binom_test(59034, n=60615, p=0.973)

In [0]:
#One-tailed Binomial test 
stats.binom_test(59034, n=60615, p=0.97, alternative="greater")

**Quadratic Discriminant Analysis**

In [0]:
sdss_qda = QuadraticDiscriminantAnalysis(tol=0.0001,reg_param=0.005)

In [0]:

qda_train, qda_test = get_stack_pred(sdss_qda)

In [0]:
from sklearn.metrics import confusion_matrix
cm_qda = confusion_matrix(Y_test, qda_test)
print(cm_qda)

In [0]:
df_qda_cm = pd.DataFrame(cm_qda,index=['Galaxy','Quasar','Star'],
                  columns = ['Galaxy','Quasar','Star'])
sb.heatmap(df_qda_cm, annot=True,fmt="g",cmap="Blues")

In [0]:
print(classification_report(Y_test, qda_test,target_names=class_names))

In [0]:
#Two-tailed binomial test to check if the results acheived were not due to chance alone
stats.binom_test(58092, n=60615, p=0.957)

In [0]:
#One tailed binomial test
stats.binom_test(58092, n=60615, p=0.956, alternative="greater")

**XGBoost**

In [0]:
xgb_model = XGBClassifier(objective="multi:softprob", random_state=18120199,n_estimators=30,n_jobs=-1,learning_rate=0.2,max_depth=10,scale_pos_weight=1,subsample=0.6,gamma=0.001,reg_alpha=0.005,min_child_weight=89)

In [0]:
xgb_model_1=XGBClassifier(objective="multi:softprob",learning_rate=0.00331,colsample_bylevel=0.6097,colsample_bynode=1,colsample_bytree=0.735,gamma=4.117,max_delta_step=0,max_depth=6,min_child_weight=89,n_estimators=5000, reg_alpha=0.169,subsample=0.602,reg_lambda=3.06,scale_pos_weight=1)

In [0]:
start=time.time()
xgb_train, xgb_test = get_stack_pred(xgb_model)
end=time.time()

In [0]:
xgb_time=end-start
print(xgb_time)

In [0]:
from sklearn.metrics import confusion_matrix
cm_xgb = confusion_matrix(Y_test, xgb_test)
print(cm_xgb)

In [0]:
df_xgb_cm = pd.DataFrame(cm_xgb,index=['Galaxy','Quasar','Star'],
                  columns = ['Galaxy','Quasar','Star'])
sb.heatmap(df_xgb_cm, annot=True,fmt="g",cmap="Blues")

In [0]:
print(classification_report(Y_test, xgb_test,target_names=class_names))

**BInomial Test**

In [0]:
stats.binom_test(59056, n=60615, p=0.973, alternative="greater") 

In [0]:
stats.binom_test(59056, n=60615, p=0.974)

**SVM**

In [0]:
sdss_svm = svm.SVC(gamma=0.4, decision_function_shape='ovr',shrinking=True,kernel='rbf',C=7.64)

In [0]:
start_svm=time.time()
svm_train, svm_test = get_stack_pred(sdss_svm)
end_svm=time.time()

In [0]:
time_svm=end_svm-start_svm

In [0]:
from sklearn.metrics import confusion_matrix
cm_svm = confusion_matrix(Y_test, svm_test)
print(cm_svm)

In [0]:
df_svm_cm = pd.DataFrame(cm_svm,index=['Galaxy','Quasar','Star'],
                  columns = ['Galaxy','Quasar','Star'])
sb.heatmap(df_svm_cm, annot=True,fmt="g",cmap="Blues")

In [0]:
print(classification_report(Y_test, svm_test,target_names=class_names))

**Binomial Test**

In [0]:
stats.binom_test(58949, n=60615, p=0.971, alternative="greater")

In [0]:
stats.binom_test(58949, n=60615, p=0.972)

**Kruskal Wallice H test**

In [0]:
from scipy import stats

In [0]:
stats.kruskal(nn_test,qda_test,svm_test,xgb_test,et_test)

**Wilcoxon sign rank test**

In [0]:
#prediction result arrays need to be flattened before conducting tests on them 
nn_f=nn_test.flatten('F')
et_f=et_test.flatten('F')
qda_f=qda_test.flatten('F')
xgb_f=xgb_test.flatten('F')
svm_f=svm_test.flatten('F')

In [0]:
#Wilcoxon signed rank test between Neural Network and Extra trees
wilcox_rank_nn_et, pvalue_nn_et = wilcoxon(nn_f,et_f)
print(wilcox_rank_nn_et)
print(pvalue_nn_et)

In [0]:
#Wilcoxon signed rank test between Neural Network and SVM
wilcox_rank_nn_svm, pvalue_nn_svm = wilcoxon(nn_f,svm_f)
print(wilcox_rank_nn_svm)
print(pvalue_nn_svm)

In [0]:
#Wilcoxon signed rank test between Neural Network and QDA
wilcox_rank_nn_qda, pvalue_nn_qda = wilcoxon(nn_f,qda_f)
print(wilcox_rank_nn_qda)
print(pvalue_nn_qda)

In [0]:
#Wilcoxon signed rank test between Neural Network and XGBoost
wilcox_rank_nn_xgb, pvalue_nn_xgb = wilcoxon(nn_f,xgb_f)
print(wilcox_rank_nn_xgb)
print(pvalue_nn_xgb)

In [0]:
#Wilcoxon signed rank test between Extra trees and QDA
wilcox_rank_et_qda, pvalue_et_qda = wilcoxon(qda_f,et_f)
print(wilcox_rank_et_qda)
print(pvalue_et_qda)

In [0]:
#Wilcoxon signed rank test between Extra trees and XGB
wilcox_rank_et_xgb, pvalue_et_xgb = wilcoxon(xgb_f,et_f)
print(wilcox_rank_et_xgb)
print(pvalue_et_xgb)

In [0]:
#Wilcoxon signed rank test between Extra trees and SVM
wilcox_rank_et_svm, pvalue_et_svm = wilcoxon(svm_f,et_f)
print(wilcox_rank_et_svm)
print(pvalue_et_svm)

In [0]:
#Wilcoxon signed rank test between XGB and SVM
wilcox_rank_xgb_svm, pvalue_xgb_svm = wilcoxon(xgb_f,svm_f)
print(wilcox_rank_xgb_svm)
print(pvalue_xgb_svm)

In [0]:
#Wilcoxon signed rank test between XGB and QDA
wilcox_rank_xgb_qda, pvalue_xgb_qda = wilcoxon(xgb_f,qda_f)
print(wilcox_rank_xgb_qda)
print(pvalue_xgb_qda)

In [0]:
#Wilcoxon signed rank test between SVM and QDA
wilcox_rank_svm_qda, pvalue_svm_qda = wilcoxon(svm_f,qda_f)
print(wilcox_rank_svm_qda)
print(pvalue_svm_qda)

**Combining Classifier Predictions**

In [0]:
train_meta=np.concatenate((nn_train,et_train,qda_train,xgb_train,svm_train,),axis=1)
test_meta=np.concatenate((nn_test,et_test,qda_test,xgb_test,svm_test),axis=1)

In [0]:
train_meta.shape

In [0]:
X_train.shape

In [0]:
Y_train.shape

In [0]:
test_meta.shape

In [0]:
Y_test.shape

**MLP as meta learner**

In [0]:
def meta_model():
  model_sdss_base = Sequential()
  model_sdss_base.add(Dense(512, input_shape=(5,)))
  model_sdss_base.add(LeakyReLU())
  model_sdss_base.add(Dropout(rate=0.2))
  model_sdss_base.add(Dense(512))
  model_sdss_base.add(LeakyReLU())
  model_sdss_base.add(Dropout(rate=0.2))
  model_sdss_base.add(Dense(256))
  model_sdss_base.add(LeakyReLU())
  adamopt=optimizers.Adam(lr=10**-3)
  model_sdss_base.add(Dense(3, activation='softmax'))
  model_sdss_base.compile(loss='categorical_crossentropy',
                    optimizer=adamopt, 
                    metrics=['accuracy'])
  return model_sdss_base

In [0]:
meta_nn_els = EarlyStopping(monitor='acc', mode='max', verbose=1, patience=5)
meta_nn_mck = ModelCheckpoint('best_observed_meta_model.h5', monitor='acc', mode='max', verbose=1, save_best_only=True)

In [0]:
nn_meta = KerasClassifier(build_fn=meta_model, epochs=50, batch_size=100, verbose=1,callbacks=[meta_nn_els,meta_nn_mck])

In [0]:
nn_meta.fit(train_meta,Y_train)

In [0]:
!ls

In [0]:
from google.colab import files
files.download('best_observed_meta_model.h5') 

In [0]:
meta_test=nn_meta.predict(test_meta)

In [0]:
meta_test=meta_test.reshape(-1,1)

In [0]:
col=['Meta-Learner-Predictions']

In [0]:
meta_predictions=pd.DataFrame(meta_test,columns=col)

In [0]:
meta_predictions.to_csv('meta_predictions.csv',index = False)

In [0]:
files.download('meta_predictions.csv')

In [0]:
print(classification_report(Y_test, meta_test))

In [0]:
from sklearn.metrics import confusion_matrix
cm_meta = confusion_matrix(Y_test, meta_test)
print(cm_meta)

In [0]:
df_meta_cm = pd.DataFrame(cm_meta,index=['Galaxy','Quasar','Star'],
                  columns = ['Galaxy','Quasar','Star'])
sb.heatmap(df_meta_cm, annot=True,fmt="g",cmap="Blues")

**Binomial Test**

In [0]:
#Two-tailed binomial test
stats.binom_test(59265, n=60615, p=0.980)

In [0]:
#One-tailed binomial test
stats.binom_test(59265, n=60615, p=0.978,alternative="greater")

**Kruskal Wallice H test**

In [0]:
from scipy import stats

In [0]:
stats.kruskal(nn_test,qda_test,svm_test,xgb_test,et_test,meta_test)

**Wilcoxon sign rank test**

In [0]:
meta_f=meta_test.flatten('F')

In [0]:
#Wilcoxon signed rank test between meta learner and Neural Networks
wilcox_rank_meta_nn, pvalue_meta_nn = wilcoxon(nn_f,meta_f)
print(wilcox_rank_meta_nn)
print(pvalue_meta_nn)

In [0]:
#Wilcoxon signed rank test between meta learner and Extra trees
wilcox_rank_meta_et, pvalue_meta_et = wilcoxon(et_f,meta_f)
print(wilcox_rank_meta_et)
print(pvalue_meta_et)

In [0]:
#Wilcoxon signed rank test between meta learner and XGB
wilcox_rank_meta_xgb, pvalue_meta_xgb = wilcoxon(xgb_f,meta_f)
print(wilcox_rank_meta_xgb)
print(pvalue_meta_xgb)

In [0]:
#Wilcoxon signed rank test between meta learner and QDA
wilcox_rank_meta_qda, pvalue_meta_qda = wilcoxon(qda_f,meta_f)
print(wilcox_rank_meta_qda)
print(pvalue_meta_qda)

In [0]:
#Wilcoxon signed rank test between meta learner and SVM
wilcox_rank_meta_svm, pvalue_meta_svm = wilcoxon(svm_f,meta_f)
print(wilcox_rank_meta_svm)
print(pvalue_meta_svm)